# Data Poising im Pytorch Model

In [ ]:
import numpy as np
import os
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("whitegrid")
from sklearn.metrics import accuracy_score

import torch

In [ ]:
# Hilfsfunktion um die vorhergesagten Wahrscheinlichkeiten in die Klassen umzuwandeln
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

In [ ]:
# Hilfsfunktion um die Daten zu visualisieren
def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    if (dark):
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha = 1, cmap=cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    if(file_name):
        plt.savefig(file_name)
        plt.close()

In [ ]:
# Größe des Datensatzes
N_SAMPLES = 1000
# Größe des Testsets
TEST_SIZE = 0.1

In [ ]:
# Generierung des Datansatzes und split in test und train set
X, y = make_moons(n_samples = N_SAMPLES, noise=0.2, random_state=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [ ]:
# Data Poising
procent = 0.5
broken = int(len(y_train)*procent)
trues = len(y_train) - broken
y_poison = np.concatenate((y_train[:trues], 1-y_train[-broken:]), axis=None)
# Visualisiung der Daten
make_plot(X_train,y_poison,"Data poison")

In [ ]:
# Check ob eine GPU da ist
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Festlegen der Architektur
model = torch.nn.Sequential(
            torch.nn.Linear(2, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, 1),
            torch.nn.Sigmoid()
)
# Festlegen der Lossfunktion
loss_fn = torch.nn.BCELoss()

# Alles rüberschieben, sodass pytorch alles übernimmt
# Pytorch will Tensoren haben
xx = torch.from_numpy(X_train).to(device,torch.float32)
yy = torch.from_numpy(y_poison).to(device,torch.float32)
yy = yy.unsqueeze(-1)

# Festlegen der Lernrate 
learning_rate = 1e-4
# Festlegen des Optimierungsalgorithmus
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
# Schleife über eine Anzahl von vorher festgelegten Epochen, um zu lernen
for t in range(10000):
    # Vorwärtsdurchlauf
    y_pred = model(xx)

    # Lossfunktion ausrechnen und anzeigen
    loss = loss_fn(y_pred, yy)
    if t % 100 == 99:
        print(t, loss.item())

    # Pytorch buffert den Gradienten aus allen vorigen Berechnungen
    # Um das los zu werden und nur den aktuellen zu haben
    # Nullen wir einmal den Gradienten
    optimizer.zero_grad()

    # Rückwärtsdurchlauf
    loss.backward()

    # Update der Parameter
    optimizer.step()

In [ ]:
# Hilfsvariablen für die Visualisierung
GRID_X_START = -1.5
GRID_X_END = 2.5
GRID_Y_START = -1.0
GRID_Y_END = 2

# Erstellen eines Gitter für das Bild
grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
grid_2d = grid.reshape(2, -1).T
XX, YY = grid

In [ ]:
# Testset zu pytorch schrieben
xx_test = torch.from_numpy(X_test).to(device,torch.float32)
# Vorwärtsdurchlauf auf dem Testset
y_hat = model(xx_test).cpu().detach().numpy()
# Umwandeln in Klassen
y_pred = convert_prob_into_class(y_hat)
acc_test = (y_test.reshape(int(N_SAMPLES*TEST_SIZE),1) == y_pred).all(axis=1).mean()
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Visualisierung auf dem Gitter
xx = torch.from_numpy(grid_2d).to(device,torch.float32)
y_plot = y_hat = model(xx).cpu().detach().numpy()
make_plot(X_test, y_test, "Pytorch Model", file_name=None, XX=XX, YY=YY, preds=y_plot)